In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
from utils import print_dict_summary
from data_funcs import load_and_fix_data, rmse
from moisture_rnn import run_case

## Original Case - Single Batch

### Reproducibility Dataset

In [ ]:
reproducibility_file='../data/reproducibility_dict.pickle'

repro={}
repro.update(load_and_fix_data(reproducibility_file))
print_dict_summary(repro)

In [ ]:
param_sets_ORIG = {'id':0,
        'purpose':'reproducibility',
        'batch_size':np.inf,
        'training':None,
        'cases':['case11'],
        'scale':0,
        'rain_do':False,
#        'verbose':False,
        'verbose':1,
        'timesteps':5,
        'activation':['linear','linear'],
        'centering':[0.0,0.0],
        'hidden_units':6,
        'dense_units':1,
        'dense_layers':1,
        'DeltaE':[0,-1],    # -1.0 is to correct E bias but put at the end
        'synthetic':False,  # run also synthetic cases
        'T1': 0.1,          # 1/fuel class (10)
        'fm_raise_vs_rain': 2.0,         # fm increase per mm rain                              
        'epochs':5000,
        'verbose_fit':0,
        'verbose_weights':False,
        'note':'check 5 should give zero error'
        }

### Using `run_case` Function

In [ ]:
reproducibility.set_seed()
print('Running reproducibility')
assert param_sets_ORIG['purpose'] == 'reproducibility'
param_sets_ORIG['initialize']=True 
case = 'case11'
case_data=repro[case]
run_case(case_data,param_sets_ORIG)

### Manual Steps

In [ ]:
from moisture_rnn import create_rnn_data_1, create_rnn_data_2, train_rnn
h2=case_data['h2']
params = param_sets_ORIG
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data_1(case_data,params)
create_rnn_data_2(rnn_dat,params)

print(rnn_dat["x_train"].shape)
print(rnn_dat["y_train"].shape)

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    params,
    rnn_dat['hours'],
    fit=True
)

## Reproduce with Stateful

*NOTE*: using fewer epochs as the training loss levels off much quicker with stateful.

In [ ]:
param_sets_STATEFUL = {'id':0,
        'purpose':'reproducibility',
        'batch_size':32,
        'training':5,
        'cases':['case11'],
        'scale':0,
        'rain_do':False,
#        'verbose':False,
        'verbose':1,
        'timesteps':5,
        'activation':['linear','linear'],
        'centering':[0.0,0.0],
        'hidden_units':6,
        'dense_units':1,
        'dense_layers':1,
        'DeltaE':[0,-1],    # -1.0 is to correct E bias but put at the end
        'synthetic':False,  # run also synthetic cases
        'T1': 0.1,          # 1/fuel class (10)
        'fm_raise_vs_rain': 2.0,         # fm increase per mm rain                              
        'epochs':500,
        'verbose_fit':0,
        'verbose_weights':False,
        'note':'check 5 should give zero error',
        'verbose': False # another verbose arg used in staircase_2
        }

### Using `run_case`

In [ ]:
reproducibility.set_seed()
param_sets_STATEFUL['initialize']=True 
case = 'case11'
case_data=repro[case]
run_case(case_data,param_sets_STATEFUL)

### Manual Steps

In [ ]:
h2=case_data['h2']
params = param_sets_STATEFUL
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data_1(case_data,params)
create_rnn_data_2(rnn_dat,params)

print(rnn_dat["x_train"].shape)
print(rnn_dat["y_train"].shape)

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import train_rnn

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    params,
    rnn_dat['hours'],
    fit=True
)

In [ ]:
from moisture_rnn import rnn_predict
preds = rnn_predict(model_predict, param_sets_STATEFUL, rnn_dat)

np.sqrt(np.sum((preds - np.squeeze(rnn_dat["Y"]))**2))

# Changing Other Hyperparams

## Stateless RNN

In [ ]:
N = len(repro[case]["fm"]) # total observations
# train_ind = int(N*.8) # index of train/val split
train_ind = 300
# test_ind = int(N*.9) # index of train/test split
test_ind = 500

X = np.vstack((repro[case]["Ed"], repro[case]["Ew"])).T
y = repro[case]["fm"]

X_train = X[:train_ind]
X_val = X[train_ind:test_ind]
X_test = X[test_ind:]
y_train = y[:train_ind].reshape(-1,1)
y_val = y[train_ind:test_ind].reshape(-1,1)
y_test = y[test_ind:].reshape(-1,1)

print(f"Total Observations: {N}")
print(f"Num Training: {X_train.shape[0]}")
print(f"Num Validation: {X_val.shape[0]}")
print(f"Num Test: {X_test.shape[0]}")

In [ ]:
params = {
        'timesteps':5, # length of sequence used in gradient calculation
        'batch_size':32, # number of sequences (of length timesteps) processed in single training pass
        'scale':0,
        'activation':['linear','linear'], # [hidden layer, output layer]
        'hidden_units':6,
        # 'dense_units':1, # for single timeseries
        # 'dense_layers':1,                           
        'epochs':100
        }

In [ ]:
from moisture_rnn import staircase
X_train, y_train = staircase(X_train, y_train, timesteps = params["timesteps"], datapoints = len(y_train))
X_val, y_val = staircase(X_val, y_val, timesteps = params["timesteps"], datapoints = len(y_val))
X_test, y_test = staircase(X_test, y_test, timesteps = params["timesteps"], datapoints = len(y_test))

In [ ]:
print(f"y_train shape: {y_train.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
reproducibility.set_seed()
model = tf.keras.Sequential()
model.add(tf.keras.layers.SimpleRNN(6, input_shape=(params["timesteps"], X_train.shape[2]), activation="linear"))
model.add(tf.keras.layers.Dense(1, activation="linear")) 
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

In [ ]:
fitted = model.predict(X_train)
preds = model.predict(X_test)
print("~"*50)
print(f"RMSE Train: {rmse(fitted, y_train)}")
print(f"RMSE Test: {rmse(preds, y_test)}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.semilogy()
plt.show()

In [ ]:
repro[case].keys()

## Changing Hyperparams

In [ ]:
reproducibility.set_seed()
model = tf.keras.Sequential()
model.add(tf.keras.layers.SimpleRNN(
    units=6, 
    input_shape=(params["timesteps"], X_train.shape[2]), 
    activation="linear",
    dropout= 0.2,
    recurrent_dropout = 0.2,
))
model.add(tf.keras.layers.Dense(1, activation="linear")) 
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
model.fit(X_train, y_train, 
          epochs=20, 
          batch_size = 16, 
          validation_data=(X_val, y_val))

fitted = model.predict(X_train)
preds = model.predict(X_test)

print(f"RMSE Train: {rmse(fitted, y_train)}")
print(f"RMSE Test: {rmse(preds, y_test)}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.semilogy()
plt.legend()
plt.show()

## Stateful RNN - Batch Size in Inputs

In [ ]:
N = len(repro[case]["fm"]) # total observations
train_ind = 300 # index of train/val split
test_ind = 500 # index of train/test split

X = np.vstack((repro[case]["Ed"], repro[case]["Ew"])).T
y = repro[case]["fm"]

X_train = X[:train_ind]
X_val = X[train_ind:test_ind]
X_test = X[test_ind:]
y_train = y[:train_ind].reshape(-1,1)
y_val = y[train_ind:test_ind].reshape(-1,1)
y_test = y[test_ind:].reshape(-1,1)

print(f"Total Observations: {N}")
print(f"Num Training: {X_train.shape[0]}")
print(f"Num Validation: {X_val.shape[0]}")
print(f"Num Test: {X_test.shape[0]}")

In [ ]:
X_train.shape

In [ ]:
batch_size = params["batch_size"]
timesteps = params["timesteps"]
features = X_train.shape[1]

reproducibility.set_seed()
model = tf.keras.Sequential()
model.add(tf.keras.layers.SimpleRNN(6, batch_input_shape=(batch_size, timesteps, features), activation="linear", stateful=True))
model.add(tf.keras.layers.Dense(1, activation="linear")) 
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
## Naive approach DOESN'T WORK
# ntrain = X_train.shape[0]
# X_train.reshape((ntrain, timesteps, features))

In [ ]:
from moisture_rnn import staircase_2
X_train, y_train = staircase_2(X_train, y_train, timesteps = params["timesteps"], batch_size=params["batch_size"], verbose=False)
X_val, y_val = staircase_2(X_val, y_val, timesteps = params["timesteps"], batch_size=params["batch_size"], verbose=False)
X_test, y_test = staircase_2(X_test, y_test, timesteps = params["timesteps"], batch_size=params["batch_size"], verbose=False)

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data = (X_val, y_val))

In [ ]:
fitted = model.predict(X_train)
preds = model.predict(X_test)

print(f"RMSE Train: {rmse(fitted, y_train)}")
print(f"RMSE Test: {rmse(preds, y_test)}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## With Keras Data Function

In [ ]:
N = len(repro[case]["fm"]) # total observations
train_ind = 300 # index of train/val split
test_ind = 500 # index of train/test split

X = np.vstack((repro[case]["Ed"], repro[case]["Ew"])).T
y = repro[case]["fm"]

X_train = X[:train_ind]
X_val = X[train_ind:test_ind]
X_test = X[test_ind:]
y_train = y[:train_ind].reshape(-1,1)
y_val = y[train_ind:test_ind].reshape(-1,1)
y_test = y[test_ind:].reshape(-1,1)

print(f"Total Observations: {N}")
print(f"Num Training: {X_train.shape[0]}")
print(f"Num Validation: {X_val.shape[0]}")
print(f"Num Test: {X_test.shape[0]}")

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

dataset=timeseries_dataset_from_array(
    X_train,
    targets=y_train,
    sequence_length=timesteps,
    batch_size=batch_size,
    shuffle=False
)

def drop_last_incomplete_batch(dataset):
    return dataset.filter(lambda x, y: tf.shape(x)[0] == batch_size)

dataset = drop_last_incomplete_batch(dataset)

# Ensure the input data has the correct shape
def preprocess(features, labels):
    features = tf.expand_dims(features, axis=-1)  # add the channel dimension
    return features, labels

dataset = dataset.map(preprocess)

In [ ]:
# Define the stateful SimpleRNN model
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(6, activation='linear', stateful=True, batch_input_shape=(batch_size, timesteps, features)),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Fit the model with shuffle=False
model.fit(dataset, epochs=200, shuffle=False)

In [ ]:
dataset_test=timeseries_dataset_from_array(
    X_test,
    targets=y_test,
    sequence_length=timesteps,
    batch_size=batch_size,
    shuffle=False
)

dataset_test = drop_last_incomplete_batch(dataset_test)
dataset_test = dataset_test.map(preprocess)

In [ ]:
fitted = model.predict(dataset).flatten()
y_train = np.concatenate([y.numpy() for _, y in dataset], axis=0)
preds = model.predict(dataset_test).flatten()
y_test = np.concatenate([y.numpy() for _, y in dataset_test], axis=0)

print("~"*50)
print(f"RMSE Train: {rmse(fitted, y_train)}")
print(f"RMSE Test: {rmse(preds, y_test)}")